<a href="https://colab.research.google.com/github/HABIBARAFIQ/AI-ML-practice-projects/blob/main/PPTGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai python-pptx Pillow requests python-dotenv

In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from pptx import Presentation
from pptx.util import Inches,Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from PIL import Image
import requests
import json
from io import BytesIO
load_dotenv()

False

In [ ]:

# just run this cell for configuring all files
class PPTGenerator:
    def __init__(self, api_key=None):
        self.api_key = api_key
        if not self.api_key:
            raise ValueError("API Key is required")
        print("Configuring genai with API key.")
        genai.configure(api_key=self.api_key)
        self.model = genai.GenerativeModel('gemini-2.5-pro')
        self.model_vision = genai.GenerativeModel('gemini-2.5-pro')
        self.presentation = Presentation()
        print("genai configured.")
    def _get_fallback_outline(self, topic, num_slides):
        """Fallback outline if Gemini fails"""
        return [
            {
                "title": f"Introduction to {topic}",
                "content": "• Overview of the topic\n• Key objectives\n• What to expect",
                "slide_type": "title"
            },
            {
                "title": "Background Information",
                "content": "• Historical context\n• Current state\n• Important facts",
                "slide_type": "content"
            },
            {
                "title": "Key Concepts",
                "content": "• Main principles\n• Core ideas\n• Essential knowledge",
                "slide_type": "content"
            },
            {
                "title": "Applications and Examples",
                "content": "• Real-world applications\n• Case studies\n• Practical examples",
                "slide_type": "content"
            },
            {
                "title": "Conclusion",
                "content": "• Summary of key points\n• Final thoughts\n• Next steps",
                "slide_type": "conclusion"
            }
        ]

    def generate_content_outline(self, topic, num_of_slides=5):
        prompt = f"""
        Create a detailed outline for a PowerPoint presentation on "{topic}" with {num_of_slides} slides.
        Return the response as a JSON array with the following structure:
        [
            {{
                "title": "Slide Title",
                "content": "Main content points as bullet points",
                "slide_type": "title|content|image|conclusion"
            }}
        ]

        Make sure the content is engaging, informative, and well-structured.
        The response must be a valid JSON array.
        """
        try:
            response = self.model.generate_content(prompt)
            print(response.text.strip())
            content = response.text.strip()
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0].strip()
            elif "```" in content:
                content = content.split("```")[1].strip()

            if not content.startswith('[') or not content.endswith(']'):
                print("Invalid JSON format received. Using fallback outline.")
                return self._get_fallback_outline(topic, num_of_slides)

            try:
                return json.loads(content)
            except json.JSONDecodeError as je:
                print(f"JSON parsing error: {je}")
                return self._get_fallback_outline(topic, num_of_slides)

        except Exception as e:
            print(f"Error generating content: {e}")
            return self._get_fallback_outline(topic, num_of_slides)

    def generate_image_description(self, slide_content):
        """Generate image description for slides using Gemini"""
        prompt = f"""
        Based on this slide content, suggest a relevant image description that would enhance the presentation:

        {slide_content}

        Return only a brief, descriptive phrase suitable for image search (max 10 words).
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text.strip()
        except:
            return "professional presentation"

    def download_image(self, query, save_path="temp_image.jpg"):
        """Download a relevant image from Pexels based on the query"""
        try:
            url = "https://api.pexels.com/v1/search"
            headers = {
                'Authorization': pexels_api_key
            }
            params = {
                'query': query,
                'per_page': 1,
                'orientation': 'landscape'
            }

            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()

            data = response.json()
            if not data.get('photos'):
                raise ValueError(f"No images found for query: {query}")
            print(data['photos'])

            image_url = data['photos'][0]['src']['original']

            img_response = requests.get(image_url)
            img_response.raise_for_status()

            with open(save_path, 'wb') as f:
                f.write(img_response.content)

            return save_path

        except Exception as e:
            print(f"Error downloading image: {e}")
            try:
                img = Image.new('RGB', (800, 600), color='#4A90E2')
                img.save(save_path)
                return save_path
            except Exception as e:
                print(f"Error creating placeholder image: {e}")
                return None

    def create_title_slide(self, title, subtitle=""):
        """Create a title slide"""
        slide_layout = self.presentation.slide_layouts[0]
        slide = self.presentation.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(44)
        title_shape.text_frame.paragraphs[0].font.bold = True
        title_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        if subtitle:
            subtitle_shape = slide.placeholders[1]
            subtitle_shape.text = subtitle
            subtitle_shape.text_frame.paragraphs[0].font.size = Pt(24)
            subtitle_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        return slide

    def create_content_slide(self, title, content, include_image=False):
        """Create a content slide with bullet points"""
        slide_layout = self.presentation.slide_layouts[1]
        slide = self.presentation.slides.add_slide(slide_layout)

        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(36)
        title_shape.text_frame.paragraphs[0].font.bold = True

        content_shape = slide.placeholders[1]
        content_shape.text = content

        text_frame = content_shape.text_frame
        for paragraph in text_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        if include_image:
            try:
                image_desc = self.generate_image_description(content)
                image_path = self.download_image(image_desc)
                if image_path and os.path.exists(image_path):
                    slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))
                    os.remove(image_path)
            except Exception as e:
                print(f"Error adding image: {e}")

        return slide

    def create_image_slide(self, title, content, image_query):
        """Create a slide with image and text"""
        slide_layout = self.presentation.slide_layouts[8]
        slide = self.presentation.slides.add_slide(slide_layout)

        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
        title_frame = title_box.text_frame
        title_frame.text = title
        title_frame.paragraphs[0].font.size = Pt(36)
        title_frame.paragraphs[0].font.bold = True
        title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        content_box = slide.shapes.add_textbox(Inches(0.5), Inches(2), Inches(4), Inches(5))
        content_frame = content_box.text_frame
        content_frame.text = content

        for paragraph in content_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        try:
            image_path = self.download_image(image_query)
            if image_path and os.path.exists(image_path):
                slide.shapes.add_picture(image_path, Inches(5), Inches(2), height=Inches(5))
                os.remove(image_path)
        except Exception as e:
            print(f"Error adding image: {e}")

        return slide

    def generate_presentation(self, topic, num_slides=5, output_path="generated_presentation.pptx"):
        """Generate a complete PowerPoint presentation"""
        print(f"Generating presentation on: {topic}")

        outline = self.generate_content_outline(topic, num_slides)

        for i, slide_data in enumerate(outline):
            title = slide_data["title"]
            content = slide_data["content"]
            slide_type = slide_data["slide_type"]

            print(f"Creating slide {i+1}: {title}")

            if i == 0 or slide_type == "title":
                self.create_title_slide(title, f"Generated by Gemini AI")
            elif slide_type == "image":
                image_query = self.generate_image_description(content)
                self.create_image_slide(title, content, image_query)
            else:
                include_image = (i % 3 == 0)
                self.create_content_slide(title, content, include_image)

        self.presentation.save(output_path)
        print(f"Presentation saved as: {output_path}")

        return output_path




In [ ]:
def _get_fallback_outline(self, topic, num_slides):
        """Fallback outline if Gemini fails"""
        return [
            {
                "title": f"Introduction to {topic}",
                "content": "• Overview of the topic\n• Key objectives\n• What to expect",
                "slide_type": "title"
            },
            {
                "title": "Background Information",
                "content": "• Historical context\n• Current state\n• Important facts",
                "slide_type": "content"
            },
            {
                "title": "Key Concepts",
                "content": "• Main principles\n• Core ideas\n• Essential knowledge",
                "slide_type": "content"
            },
            {
                "title": "Applications and Examples",
                "content": "• Real-world applications\n• Case studies\n• Practical examples",
                "slide_type": "content"
            },
            {
                "title": "Conclusion",
                "content": "• Summary of key points\n• Final thoughts\n• Next steps",
                "slide_type": "conclusion"
            }
        ]

In [ ]:
import os
from dotenv import load_dotenv

# Create or overwrite the .env file
with open(".env", "w") as f:
    f.write("GEMINI_API_KEY=your_gemini_api_key\n") # Replace YOUR_API_KEY_HERE with your actual API key
    f.write("PEXELS_API_KEY=your_pexels_api_key")
print(".env file created with a placeholder API key.")

# Load environment variables from .env file
load_dotenv()

# Read the API key
gemini_api_key = os.getenv("GEMINI_API_KEY")
pexels_api_key = os.getenv("PEXELS_API_KEY")


if gemini_api_key and pexels_api_key:
    print(f"API Key read from .env: {gemini_api_key} and {pexels_api_key}")
elif gemini_api_key:
    print(f"Gemini API Key read from .env: {gemini_api_key}, Pexels API Key not found.")
elif pexels_api_key:
    print(f"Pexels API Key read from .env: {pexels_api_key}, Gemini API Key not found.")
else:
    print("API Keys not found in .env file.")

.env file created with a placeholder API key.
API Key read from .env: and 


In [ ]:
def generate_content_outline(self,topic,num_of_slides=5):
  prompt = f"""
        Create a detailed outline for a PowerPoint presentation on "{topic}" with {num_of_slides} slides.
        Return the response as a JSON array with the following structure:
        [
            {{
                "title": "Slide Title",
                "content": "Main content points as bullet points",
                "slide_type": "title|content|image|conclusion"
            }}
        ]

        Make sure the content is engaging, informative, and well-structured.
        The response must be a valid JSON array.
        """
  try:
    # GENERATE CONTENT BASED ON PROMPT
      response = self.model.generate_content(prompt)
      # removes leading and trailing whitespace characters from a string. This includes
      # Spaces
      # Newlines (\n)
      # Tabs (\t)
      # Carriage returns (\r)
      print(response.text.strip())
      content = response.text.strip()
      if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
      elif "```" in content:
            content = content.split("```")[1].strip()

            # Ensure we have a valid JSON string
      if not content.startswith('[') or not content.endswith(']'):
            print("Invalid JSON format received. Using fallback outline.")
            return self._get_fallback_outline(topic, num_of_slides)

      try:
            return json.loads(content)
      except json.JSONDecodeError as je:
            print(f"JSON parsing error: {je}")
            return self._get_fallback_outline(topic, num_of_slides)

  except Exception as e:
            print(f"Error generating content: {e}")
            return self._get_fallback_outline(topic, num_of_slides)


In [ ]:
def generate_image_description(self, slide_content):
        """Generate image description for slides using Gemini"""
        prompt = f"""
        Based on this slide content, suggest a relevant image description that would enhance the presentation:

        {slide_content}

        Return only a brief, descriptive phrase suitable for image search (max 10 words).
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text.strip()
        except:
            return "professional presentation"

In [ ]:
def download_image(self, query, save_path="temp_image.jpg"):
        """Download a relevant image from Pexels based on the query"""
        try:
            # Pexels API endpoint
            url = "https://api.pexels.com/v1/search"

            # Headers with API key,inside Authorization
            headers = {
                'Authorization': 'FwPKrWnkL5BBk39FxPZcnFYOympqu9Y4ab8AzPGA7kO61CI3Ftg1tBWv'
            }

            # Search parameters
            params = {
                'query': query,
                'per_page': 1,
                'orientation': 'landscape'
            }

            # Get search results
            response = requests.get(url, headers=headers, params=params)
            # Raises an error if the HTTP request failed (i.e., returned a 4xx or 5xx status code).
            response.raise_for_status()

            # parse the response as JSON and return a Python dict
            data = response.json()
            if not data.get('photos'):
                raise ValueError(f"No images found for query: {query}")
            print(data['photos'])

            # Get the first photo's original size URL
            image_url = data['photos'][0]['src']['original']

            # Download the image
            img_response = requests.get(image_url)
            img_response.raise_for_status()

            # Save the image
            # wb' means:
            # w: Write mode (it will create or overwrite the file)
            # b: Binary mode (needed for images, not text)
            with open(save_path, 'wb') as f:
              # img_response.content is the raw image data in bytes from the HTTP response.
                f.write(img_response.content)

            return save_path

        except Exception as e:
            print(f"Error downloading image: {e}")
            # Fallback to placeholder if there's an error
            try:
                img = Image.new('RGB', (800, 600), color='#4A90E2')
                img.save(save_path)
                return save_path
            except Exception as e:
                print(f"Error creating placeholder image: {e}")
                return None



In [ ]:
def create_title_slide(self, title, subtitle=""):
        """Create a title slide"""
        slide_layout = self.presentation.slide_layouts[0]  # Title slide layout
        slide = self.presentation.slides.add_slide(slide_layout)

        # Set title
        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(44)
        title_shape.text_frame.paragraphs[0].font.bold = True
        title_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        # Set subtitle if provided
        if subtitle:
            subtitle_shape = slide.placeholders[1]
            subtitle_shape.text = subtitle
            subtitle_shape.text_frame.paragraphs[0].font.size = Pt(24)
            subtitle_shape.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        return slide

In [ ]:
def create_content_slide(self, title, content, include_image=False):
        """Create a content slide with bullet points"""
        slide_layout = self.presentation.slide_layouts[1]  # Title and content layout
        slide = self.presentation.slides.add_slide(slide_layout)

        # Set title
        title_shape = slide.shapes.title
        title_shape.text = title
        title_shape.text_frame.paragraphs[0].font.size = Pt(36)
        title_shape.text_frame.paragraphs[0].font.bold = True

        # Set content
        content_shape = slide.placeholders[1]
        content_shape.text = content

        # Format bullet points
        text_frame = content_shape.text_frame
        for paragraph in text_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        # Add image if requested
        if include_image:
            try:
                image_desc = self.generate_image_description(content)
                image_path = self.download_image(image_desc)
                if image_path and os.path.exists(image_path):
                    slide.shapes.add_picture(image_path, Inches(6), Inches(2), height=Inches(4))
                    # Clean up temporary image
                    os.remove(image_path)
            except Exception as e:
                print(f"Error adding image: {e}")

        return slide

In [ ]:
def create_image_slide(self, title, content, image_query):
        """Create a slide with image and text"""
        slide_layout = self.presentation.slide_layouts[8]  # Blank layout
        slide = self.presentation.slides.add_slide(slide_layout)

        # Add title
        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1))
        title_frame = title_box.text_frame
        title_frame.text = title
        title_frame.paragraphs[0].font.size = Pt(36)
        title_frame.paragraphs[0].font.bold = True
        title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        # Add content
        content_box = slide.shapes.add_textbox(Inches(0.5), Inches(2), Inches(4), Inches(5))
        content_frame = content_box.text_frame
        content_frame.text = content

        # Format content
        for paragraph in content_frame.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(51, 51, 51)

        # Add image
        try:
            image_path = self.download_image(image_query)
            if image_path and os.path.exists(image_path):
                slide.shapes.add_picture(image_path, Inches(5), Inches(2), height=Inches(5))
                os.remove(image_path)
        except Exception as e:
            print(f"Error adding image: {e}")

        return slide

In [ ]:
def generate_presentation(self, topic, num_slides=5, output_path="generated_presentation.pptx"):
        """Generate a complete PowerPoint presentation"""
        print(f"Generating presentation on: {topic}")

        # Generate content outline
        outline = self.generate_content_outline(topic, num_slides)

        # Create slides based on outline
        for i, slide_data in enumerate(outline):
            title = slide_data["title"]
            content = slide_data["content"]
            slide_type = slide_data["slide_type"]

            print(f"Creating slide {i+1}: {title}")

            if i == 0 or slide_type == "title":
                self.create_title_slide(title, f"Generated by Gemini AI")
            elif slide_type == "image":
                image_query = self.generate_image_description(content)
                self.create_image_slide(title, content, image_query)
            else:
                include_image = (i % 3 == 0)  # Add image every 3rd slide
                self.create_content_slide(title, content, include_image)

        # Save presentation
        self.presentation.save(output_path)
        print(f"Presentation saved as: {output_path}")

        return output_path

print("✅ PPTGenerator class defined successfully!")

✅ PPTGenerator class defined successfully!


In [ ]:
try:
    generator = PPTGenerator(api_key=gemini_api_key)
    print("✅ PPT Generator initialized successfully!")
except ValueError as e:
    print(f"❌ Error: {e}")
    print("Please set your GEMINI_API_KEY first.")

Configuring genai with API key.
genai configured.
✅ PPT Generator initialized successfully!


In [ ]:
topic = "Artificial Intelligence in Healthcare"  # Change this to your desired topic
num_slides = 6  # Change this to your desired number of slides

try:
    output_file = generator.generate_presentation(topic, num_slides, "ai_healthcare_presentation.pptx")
except Exception as e:
    print(e)

Generating presentation on: Artificial Intelligence in Healthcare
```json
[
    {
        "title": "Artificial Intelligence in Healthcare: Revolutionizing Patient Care",
        "content": "Harnessing Data to Build a Healthier Future\n- Presenter's Name\n- Date",
        "slide_type": "title"
    },
    {
        "title": "What is AI in Healthcare?",
        "content": "Definition: The use of complex algorithms and software to emulate human cognition in the analysis of complicated medical data.\n- Core Goal: To analyze relationships between prevention/treatment techniques and patient outcomes.\n- Key Technologies: Machine Learning (ML), Natural Language Processing (NLP), Computer Vision, and Robotics.\n- Why Now?: Explosion of medical data (EHRs, genomics, imaging), increased computing power, and advanced algorithms.",
        "slide_type": "content"
    },
    {
        "title": "Transformative Applications of AI",
        "content": "- Diagnostic Imaging: AI algorithms detecting dise

In [ ]:
# get link of the ppt


In [ ]:
from google.colab import files

files.download('ai_healthcare_presentation.pptx')